### Imports

In [29]:
#######################
#       IMPORTS       #
#######################
import pandas as pd
import os
import shutil
import math
import copy
import re
import sys
import getopt
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import getpass

from scipy.interpolate import interp1d
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import joblib
from shutil import copyfile
from scipy.integrate import simps

outputDataList = []

TSV_IMPACT = 24.5
TSV_FREEFALL = 5.8
TSV_CALMZONE_HIGH = 11
TSV_CALMZONE_MEDIUM = 10
TSV_CALMZONE_LOW = 8
TSV_FREEFALL_SPIN = 4

DTSV_IMPACT_LOW = 10
DTSV_IMPACT_MEDIUM = 14.7
DTSV_IMPACT_HIGH = 20


### Global Variables

In [30]:
#######################
#       SCALARS       #
#######################
EVENT_IMPACT = 1
EVENT_FREEFALL = 2

TYPE_UNKNOWN = 0
TYPE_FREEFALL_SIGNIFICANT = 1
TYPE_FREEFALL_INSIGNIFICANT = 2
TYPE_IMPACT_NEGLIGIBLE = 1
TYPE_IMPACT_SOFT = 2
TYPE_IMPACT_MEDIUM = 3
TYPE_IMPACT_HARD = 4
TYPE_IMPACT_FORCE = 5

TSV_IMPACT = 24.5
TSV_FREEFALL = 5.8
TSV_CALMZONE_HIGH = 11
TSV_CALMZONE_MEDIUM = 10
TSV_CALMZONE_LOW = 8
TSV_FREEFALL_SPIN = 4

DTSV_IMPACT_LOW = 10
DTSV_IMPACT_MEDIUM = 14.7
DTSV_IMPACT_HIGH = 20

interpolateFreq = 10
resampleFactor = 2

ZONE_PREFALL = 1
ZONE_FREEFALL = 2
ZONE_IMPACT = 3
ZONE_NOISE = 4
ZONE_CALM = 0

PREFALL_LENGTH = 200
PREIMPACT_LENGTH = 100
FREEFALL_SIGNIFICANT = 200
CALMZONE_DURATION = 30
IMPACT_LENGTH_MIN = 60
FORCE_AREA_MIN = 600
EVENT_GAP_MIN = 30

prefallTypes = [
    ["FFXP","a freefall on its left side"],
    ["FFXM","a freefall on its right side"],
    ["FFYP","a freefall on its bottom"],
    ["FFYM","a freefall on its top"],
    ["FFZP","a freefall while facing down"],
    ["FFZM","a freefall while facing up"],
    ["HFXP","a freefall following a push on its left side"],
    ["HFXM","a freefall following a push on its right side"],
    ["HFYP","a freefall following a push on its bottom"],
    ["HFYM","a freefall following a push on its top"],
    ["HFZP","a freefall following a push on its face"],
    ["HFZM","a freefall following a push on its back"],
    ["OTXX","an overhand throw"],
    ["UTXX","an underhand throw"],
    ["PFXX","a slippage while being put into pocket"],
    ["PPXX","a slippage while being put into pocket"],
    ["TCXX","an upward throw"]
    ]

impactTypes = [
    ["DSXX","was slammed on the desk"],
    ["WSXX","hit a wall"],
    ["FSXX","was slammed on the floor"]
    ]

imageTypes = [
    ["FFXP","img/DAFall.png"],
    ["FFXM","img/DAFall.png"],
    ["FFYP","img/DAFall.png"],
    ["FFYM","img/DAFall.png"],
    ["FFZP","img/DAFall.png"],
    ["FFZM","img/DAFall.png"],
    ["HFXP","img/DATableFall.png"],
    ["HFXM","img/DATableFall.png"],
    ["HFYP","img/DATableFall.png"],
    ["HFYM","img/DATableFall.png"],
    ["HFZP","img/DATableFall.png"],
    ["HFZM","img/DATableFall.png"],
    ["OTXX","img/DAOThrow.png"],
    ["UTXX","img/DAUThrow.png"],
    ["PFXX","img/DAPocketSlip.png"],
    ["PPXX","img/DAPocketSlip.png"],
    ["TCXX","img/DAThrowCatch.png"],
    ["DSXX","img/DADeskSlam.png"],
    ["WSXX","img/DAWallSlam.png"],
    ["FSXX","img/DAFloorSlam.png"]
    ]

outputDataList = []

### Helper Methods



In [31]:
#######################
#       HELPERS       #
#######################
def addEvent (eventType, eventStart, eventEnd):
    # Create new event
    processedEvent.append([eventType, False, 0, 0, eventStart, eventEnd, 0, 0, ""])
    
    currentIndex = len(processedEvent) - 1

    # Detect duration

    # Detect significance

    # 
    return currentIndex

def findEvent (eventType, eventStart):
    eventIndex = 0

    return int(eventIndex)

def updateEventValue (eventIndex, updateIndex, newValue):

    return

def updateEventCause (eventIndex, newCause):
    return updateEventValue(eventIndex, INDEX_EVENT_CAUSE, newCause)

"""
This function fills in missing data for prefall and preimpact analysis
    Input:
        (1: list) Actual dataset
        (2: int) Start of data
        (3: int) End of data
        (4: int) Required length
    Output:
        (1: list) Filled-in dataset
"""
def fillData (origDataset, start, end, length):
    finalDataset = list()
    if (length > (end - start)):
        fillNeed = length - (end - start)
    else:
        fillNeed = 0
    for i in range(fillNeed):
        finalDataset.append(origDataset[start])
    for i in range(start, end):
        finalDataset.append(origDataset[i])

    return finalDataset

"""
This function estimates fall height from fall duration.
    Input:
        (1: float) Fall duration in ms
    Output:
        (1: float) Estimated fall height in feet
"""
def estimateDistance (durarion = 0):
    return round((3.28*(9.81*(durarion/1000)*(durarion/1000))/2) * 1.225, 2)

"""
This function detects spin in a freefall.
    Input:
        (1: list) TSV dataset
        (2: int) Freefall start index
        (3: int) Freefall end index
    Output:
        (1: Bool) True if spin detected, False otherwise
"""
def detectSpin (tsvDataset, start, end):
    detected = False
    for i in range(start, end):
        if (tsvDataset[i] >= TSV_FREEFALL_SPIN):
            detected = True
            break
    
    return detected

"""
This function detects impact orientation.
    Input:
        (1: list) TSV dataset
        (2: list) X dataset
        (3: list) Y dataset
        (4: list) Z dataset
        (5: int) Impact start index
        (6: int) Impact end index
    Output:
        (1: list) Relative components (in %) of TSV in each 3 direction
"""
def detectImpactDirection (tsvDataset, xDataset, yDataset, zDataset, start, end):
    xComponent = 0
    yComponent = 0
    zComponent = 0

    maxTsv = -1
    maxI = -1

    for i in range(start, end):
        if (tsvDataset[i] > maxTsv):
            maxTsv = tsvDataset[i]
            maxI = i

    if (maxI >= 0):

        # Values
        xComponent = round((xDataset[maxI] * xDataset[maxI] * 100) / (tsvDataset[maxI] * tsvDataset[maxI]), 2)
        yComponent = round((yDataset[maxI] * yDataset[maxI] * 100) / (tsvDataset[maxI] * tsvDataset[maxI]), 2)
        zComponent = round((zDataset[maxI] * zDataset[maxI] * 100) / (tsvDataset[maxI] * tsvDataset[maxI]), 2)

        # Signs
        if (xDataset[maxI] < 0):
            xComponent = -xComponent

        if (yDataset[maxI] < 0):
            yComponent = -yComponent

        if (zDataset[maxI] < 0):
            zComponent = -zComponent
        
    return [xComponent, yComponent, zComponent]

"""
This function merges closely spaced similar events to handle in-event noises.
    Input:
        (1: list) Events' list
    Output:
        (1: list) Merged events' list
"""
def mergeEvents(events):
    mergedEvents = list()

    if (len(events) <= 1):
        return events

    i = 0
    continued = False
    while (i < len(events)):
        if (i == (len(events) - 1)):
            # We're at last event
            if not (continued):
                currentEvent = events[i]
            else:
                i -= 1
                continued = False
        else:
            if (continued):
                nextEvent = events[i]
            else:
                currentEvent = events[i]
                nextEvent = events[i + 1]
                i += 1

            if (nextEvent[0] == currentEvent[0]):
                # The two events are same
                if ((nextEvent[1] - currentEvent[2]) < EVENT_GAP_MIN):
                    # Too closely spaced, merge
                    currentEvent[2] = nextEvent[2]
                    if (currentEvent[0] == EVENT_IMPACT):
                        currentEvent[3] = max(currentEvent[3], nextEvent[3])
                        currentEvent[4] = max(currentEvent[4], nextEvent[4])
                        currentEvent[5] = max(currentEvent[5], nextEvent[5])
                    elif (currentEvent[0] == EVENT_FREEFALL):
                        currentEvent[3] = min(currentEvent[3], nextEvent[3])
                        currentEvent[4] = currentEvent[4] or nextEvent[4]
                    continued = True
                else:
                    continued = False
                    i -= 1
            else:
                continued = False
                i -= 1
        i += 1
        if not (continued):
            mergedEvents.append(currentEvent)
    return mergedEvents

"""
This function parses a list of events and prints the results
    Input:
        (1: list) Event list
        (2: list) Timestamp dataset
    Output:
        Nothing
"""
def parseEvents (eventList, tsDataset):
    if (debugMode):
        for event in eventList:
            print(event)

    lastEvent = 0
    for event in eventList:
        if (event[0] == EVENT_FREEFALL):
            if (event[4]):
                spinResult = "Yes"
            else:
                spinResult = "No"
            print("After", format(event[1] - lastEvent, '.2f'), "ms:", "Freefall of duration", format(tsDataset[event[2]] - tsDataset[event[1]], '.2f'), "ms, minimum TSV:", format(event[3], '.2f'), "m/s2, estimated fall:", estimateDistance(tsDataset[event[2]] - tsDataset[event[1]]), "feet, spin detected:", spinResult)
        elif(event[0] == EVENT_IMPACT):
            if (event[5] == TYPE_IMPACT_HARD):
                impactType = "Severe"
            elif (event[5] == TYPE_IMPACT_MEDIUM):
                impactType = "Medium"
            elif (event[5] == TYPE_IMPACT_SOFT):
                impactType = "Low"
            elif (event[5] == TYPE_IMPACT_FORCE):
                impactType = "Force"
            else:
                impactType = "Negligible"
            print("After", format(event[1] - lastEvent, '.2f'), "ms:", "Impact of duration", format(tsDataset[event[2]] - tsDataset[event[1]], '.2f'), "ms, maximum TSV:", format(event[3], '.2f'), "m/s2, maximum DTSV:", event[4], ", type:", impactType)
            print("Impact direction =", detectImpactDirection (tsvResampled, xResampled, yResampled, zResampled, event[1], event[2]))
        else:
            print("After", format(event[1] - lastEvent, '.2f'), "ms:", "Unknown event of duration", format(tsDataset[event[2]] - tsDataset[event[1]], '.2f'), "ms")
        lastEvent = event[2]
    return

def prefallProcess(xData, yData, zData, zoneStart, zoneEnd):
    finalDF = pd.DataFrame()
    
    rowCount = zoneEnd - zoneStart
    count = 0
       
    if (rowCount >= PREFALL_LENGTH):
        start = 0
        end = rowCount
       
        for i in range(start, end):
            temp = pd.DataFrame(columns=['X'+str(i), 'Y'+str(i), 'Z'+str(i)])
            temp.loc[0] = [xData[i], yData[i], zData[i]]
            temp = temp.reset_index(drop=True)
           
            if count == 0:
                tempDF = temp
                count = 1
            else:
                tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
               
        finalDF = finalDF.append(tempDF)
    elif (rowCount >= PREFALL_LENGTH / 2):
        # Not enough data, but still try with padding
        diff = PREFALL_LENGTH - rowCount
        start = 0
        end = PREFALL_LENGTH
        countDown = PREFALL_LENGTH - rowCount
        
        for i in range(start, end):
            temp = pd.DataFrame(columns=['X'+str(i), 'Y'+str(i), 'Z'+str(i)])
            if (countDown > 0):
                temp.loc[0] = [xData[0], yData[0], zData[0]]
                countDown -= 1
            else:
                temp.loc[0] = [xData[i - diff], yData[i - diff], zData[i - diff]]
            temp = temp.reset_index(drop=True)
           
            if count == 0:
                tempDF = temp
                count = 1
            else:
                tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
               
        finalDF = finalDF.append(tempDF)
    else:
        # Not nearly enough data
        print("Cannot do prefall analysis. Number of samples is less than", PREFALL_LENGTH)
    return finalDF

def preimpactProcess(xData, yData, zData, zoneStart, zoneEnd):
    finalDF = pd.DataFrame()
    
    rowCount = zoneEnd - zoneStart
    count = 0
       
    if (rowCount >= PREIMPACT_LENGTH):
        start = 0
        end = rowCount
       
        for i in range(start, end):
            temp = pd.DataFrame(columns=['X'+str(i), 'Y'+str(i), 'Z'+str(i)])
            temp.loc[0] = [xData[i], yData[i], zData[i]]
            temp = temp.reset_index(drop=True)
           
            if count == 0:
                tempDF = temp
                count = 1
            else:
                tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
               
        finalDF = finalDF.append(tempDF)
    elif (rowCount >= PREIMPACT_LENGTH / 2):
        # Not enough data, but still try with padding
        diff = PREIMPACT_LENGTH - rowCount
        start = 0
        end = PREIMPACT_LENGTH
        countDown = PREIMPACT_LENGTH - rowCount
        
        for i in range(start, end):
            temp = pd.DataFrame(columns=['X'+str(i), 'Y'+str(i), 'Z'+str(i)])
            if (countDown > 0):
                temp.loc[0] = [xData[0], yData[0], zData[0]]
                countDown -= 1
            else:
                temp.loc[0] = [xData[i - diff], yData[i - diff], zData[i - diff]]
            temp = temp.reset_index(drop=True)
           
            if count == 0:
                tempDF = temp
                count = 1
            else:
                tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
               
        finalDF = finalDF.append(tempDF)
    else:
        # Not nearly enough data
        print("Cannot do prefall analysis. Number of samples is less than", PREIMPACT_LENGTH)
    return finalDF

def convertEventCode(eventCode):
    for event in prefallTypes:
        if (event[0] == eventCode):
            return event[1]

    for event in impactTypes:
        if (event[0] == eventCode):
            return event[1]

    return "Unknown event"

def getImageName(eventCode):
    for event in imageTypes:
        if (event[0] == eventCode):
            return event[1]

    return "Unknown.png"

def detectType(eventList, fallNum, impactNum, fallType, impactTypes):
    impactResult = ""
    freefallResult = ""
    result = "The device"
    firstFall = True
    firstImpact = True
    imageFileName = "img/Unknown.png"

    if (fallNum + impactNum == 0):
        result = result + " did not suffer any major abuse"
        impactResult = "No significant impacts detected"
        freefallResult = "No significant freefalls detected"
        return freefallResult, impactResult, result, imageFileName
    
    if (fallNum == 0):
        result = result + " did not have a significant fall"
    elif (fallNum == 1):
        if (len(fallType) == 0):
            result = result + " fell down once"
        else:
            result = result + " suffered " + convertEventCode(fallType)
            imageFileName = getImageName(fallType)
    else:
        if (len(fallType) == 0):
            result = result + " fell down " + str(fallNum) + " time(s)"
        else:
            result = result + " suffered " + convertEventCode(fallType)
            imageFileName = getImageName(fallType)

    if (impactNum == 0):
        result = result + ", but did not hit anything hard."
    else:
        if (fallNum == 0):
            result = result + ", but "
            if(len(impactTypes[0]) > 0):
                result = result + convertEventCode(impactTypes[0])
                imageFileName = getImageName(impactTypes[0])
        else:
            result = result + ", and "
            if (impactNum == 1):
                result = result + "hit a surface."
            elif (impactNum == 2):
                result = result + "there were a couple of impacts."
            elif (impactNum > 2):
                result = result + "there were multiple impacts (" + str(impactNum) + ")"       

    for event in eventList:
        if (event[0] == EVENT_FREEFALL):
            if (event[4]):
                spinResult = "Yes"
            else:
                spinResult = "No"
            if (firstFall):
                firstFall = False
            else:
                freefallResult = freefallResult + "<br><br>"
            freefallResult = freefallResult + "<b>Freefall</b> of duration " + str(round(tsResampled[event[2]] - tsResampled[event[1]], 2)) + " ms<br>Minimum TSV: " + str(round(event[3], 2))  + " m/s2<br>Estimated fall height: " + str(round(estimateDistance(tsResampled[event[2]] - tsResampled[event[1]]), 2)) + " feet<br>Spin detected: " + spinResult
        elif (event[0] == EVENT_IMPACT):
            if (firstImpact):
                firstImpact = False
            else:
                impactResult = impactResult + "<br><br>"
            if (event[5] == TYPE_IMPACT_FORCE):
                impactResult = impactResult + "<b>Force</b> impartion of duration " + str(round(tsResampled[event[2]] - tsResampled[event[1]], 2)) + " ms<br>Maximum TSV: " + str(round(event[3], 2)) + " m/s2"
            else:
                if (event[5] == TYPE_IMPACT_HARD):
                    impactType = "Severe"
                elif (event[5] == TYPE_IMPACT_MEDIUM):
                    impactType = "Medium"
                elif (event[5] == TYPE_IMPACT_SOFT):
                    impactType = "Low"
                else:
                    impactType = "Negligible"
                impactResult = impactResult + "<b>Impact</b> of duration " + str(round(tsResampled[event[2]] - tsResampled[event[1]], 2)) + " ms<br>Maximum TSV: " + str(round(event[3], 2)) + " m/s2<br>Type: " + impactType

    if (len(impactResult) == 0):
        impactResult = "No significant impacts detected"

    if (len(freefallResult) == 0):
        freefallResult = "No significant freefalls detected"

    #print(freefallResult)
    #print(impactResult)
    #print(result)

    return freefallResult, impactResult, result, imageFileName


#### Detect Events

In [32]:
"""
This function detects freefall and impact events in TSV, DTSV datasets.
    Input:
        (1: list) Timestamp dataset
        (2: list) TSV dataset
        (3: list) DTSV dataset
    Output:
        (1: list) Detected events
                    Freefall: [[EVENT_FREEFALL, eventStart, eventEnd, minTsv, spinFlag]]
                    Impact:   [[EVENT_IMPACT, eventStart, eventEnd, maxTsv, maxDtsv, impactType]]
        (2: list) Noise zones
                    [[noiseStart, noiseEnd]]
    
"""
import pdb; 

def detectEvents (tsDataset, tsvDataset, dtsvDataset):
    noiseZones = list()
    detectedEvents = list()
    numberOfSamples = len(tsDataset)

    maxTsv = -1.0
    minTsv = -1.0
    freefallStart = -1
    impactStart = -1
    noiseStart = -1
    calmZoneCount = 0
    timeDiffInMs = 0
    tsvResampled = list()
    dtsvResampled = list()

    for i in range(numberOfSamples):
        currentTsv = tsvDataset[i]
        # Update max/min TSV values if required
        if (maxTsv >= 0):
            if (maxTsv < currentTsv):
                maxTsv = currentTsv
        elif (minTsv >= 0):
            if (minTsv > currentTsv):
                minTsv = currentTsv

        if (currentTsv > TSV_IMPACT):
            # Impact zone
            if (noiseStart >= 0):
                calmZoneCount = 0
            # Finalize stuff if this marks the end of a freefall event
            if (freefallStart > 0):
                spinDetected = detectSpin(tsvDataset, freefallStart, i)
                detectedEvents.append([EVENT_FREEFALL, freefallStart, i, minTsv, spinDetected])
                freefallStart = -1
                minTsv = -1.0
            # Initialize stuff if this is the start of impact
            if (impactStart < 0):
                impactStart = i
                maxTsv = currentTsv
        elif (currentTsv < TSV_FREEFALL):
            # Freefall zone
            if (noiseStart >= 0):
                calmZoneCount = 0
            # Finalize stuff if this marks the end of an impact event
            if (impactStart > 0):
                # Look at DTSV to determine type
                maxDtsv = -1.0
                for j in range(impactStart, i):
                    if (maxDtsv < dtsvDataset[j]):
                        maxDtsv = dtsvDataset[j]
                if (maxDtsv >= DTSV_IMPACT_HIGH):
                    # If the DTSV is too highm it is an impact
                    impactType = TYPE_IMPACT_HARD
                else:
                    # Otherwise, it might be force impartion or impact
                    areaUnderCurve = simps(tsvDataset[impactStart:i], dx=timeDiffInMs)
                    print("Area under curve:", areaUnderCurve)
                    if (areaUnderCurve >= FORCE_AREA_MIN):
                        # Not actually an impact, just external application of force
                        impactType = TYPE_IMPACT_FORCE
                    else:
                        # Impact it is
                        if (maxDtsv >= DTSV_IMPACT_MEDIUM):
                            impactType = TYPE_IMPACT_MEDIUM
                        elif (maxDtsv >= DTSV_IMPACT_LOW):
                            impactType = TYPE_IMPACT_SOFT
                        else:
                            impactType = TYPE_IMPACT_NEGLIGIBLE
                detectedEvents.append([EVENT_IMPACT, impactStart, i, maxTsv, maxDtsv, impactType])

                # Noise filtering to be done only for high and medium impacts
                if ((impactType == TYPE_IMPACT_HARD) or (impactType == TYPE_IMPACT_MEDIUM)):
                    # End of impact, detect noise zone
                    if (noiseStart < 0):
                        noiseStart = i
                    calmZoneCount = 0
                impactStart = -1
                maxTsv = -1.0
            # Initialize stuff if this is the start of freefall
            if (freefallStart < 0):
                freefallStart = i
                minTsv = currentTsv
        else:
            # Regular zone
            # Check for end of noise
            if (noiseStart >= 0):
                if ((currentTsv <= TSV_CALMZONE_HIGH) and (currentTsv >= TSV_CALMZONE_LOW)):
                    calmZoneCount += 1
                    if (calmZoneCount >= CALMZONE_DURATION):
                        # We have calmed down enough
                        noiseZones.append([noiseStart, i])
                        noiseStart = -1
                        calmZoneCount = 0
                else:
                    # Still some noise
                    calmZoneCount = 0
            # Finalize stuff if this marks the end of an impact event
            if (impactStart > 0):
                # Look at DTSV to determine type
                maxDtsv = -1.0
                for j in range(impactStart, i):
                    if (maxDtsv < dtsvDataset[j]):
                        maxDtsv = dtsvDataset[j]
                if (maxDtsv >= DTSV_IMPACT_HIGH):
                    # If the DTSV is too high, it is an impact
                    impactType = TYPE_IMPACT_HARD
                else:
                    areaUnderCurve = simps(tsvDataset[impactStart:i], dx=timeDiffInMs)
                    print("Area under curve:", areaUnderCurve)
                    if (areaUnderCurve >= FORCE_AREA_MIN):
                        # Not actually an impact, just external application of force
                        impactType = TYPE_IMPACT_FORCE
                    else:
                        # Impact it is
                        if (maxDtsv >= DTSV_IMPACT_MEDIUM):
                            impactType = TYPE_IMPACT_MEDIUM
                        elif (maxDtsv >= DTSV_IMPACT_LOW):
                            impactType = TYPE_IMPACT_SOFT
                        else:
                            impactType = TYPE_IMPACT_NEGLIGIBLE
                detectedEvents.append([EVENT_IMPACT, impactStart, i, maxTsv, maxDtsv, impactType])

                # Noise filtering to be done only for high and medium impacts
                if ((impactType == TYPE_IMPACT_HARD) or (impactType == TYPE_IMPACT_MEDIUM)):
                    # End of impact, detect noise zone
                    if (noiseStart < 0):
                        noiseStart = i
                    calmZoneCount = 0
                impactStart = -1
                maxTsv = -1
            # Finalize stuff if this marks the end of a freefall event
            if (freefallStart > 0):
                spinDetected = detectSpin(tsvDataset, freefallStart, i)
                detectedEvents.append([EVENT_FREEFALL, freefallStart, i, minTsv, spinDetected])
                freefallStart = -1
                minTsv = -1
    print("Detect Event Count: "+str(len(detectedEvents)))
    print("Noise Count: " +str(len(noiseZones)))
    return detectedEvents, noiseZones

### DataFrame generation for model imput

In [33]:
def prepareTFLiteModelInput(events):
    for event in events:
        print(len(event))
        print(event)
    impactEventCount = 0
    freeFallEventCount = 0
    maxTSV = 0.0
    totalTSV = 0.0
    avgTSV = 0.0
    maxDTSV = 0.0
    totalDTSV = 0.0
    avgDTSV = 0.0
    totalSeverity = 0.0
    avgSeverity = 0.0
    minTSV = 1000.0
    totalMinTSV = 0.0
    avgMinTSV = 0.0
    totalSpinDetectedEventCount = 0
    avgSpin = 0.0

    for event in events:

        if(len(event)==6):
            impactEventCount +=1

            totalTSV += event[3]
            if (event[3] > maxTSV):
                maxTSV = event[3]

            totalDTSV += abs(event[4])
            if (abs(event[4]) > maxDTSV):
                maxDTSV = abs(event[4])

            totalSeverity += event[5] / 5.0

        elif (len(event)==5):
            freeFallEventCount +=1

            totalMinTSV += event[3]
            if (event[3] < minTSV):
                minTSV = event[3]
        
            if (event[4]):
                totalSpinDetectedEventCount +=1
            
        if (impactEventCount == 0):
            impactEventCount = 1
        avgTSV = totalTSV / impactEventCount
        avgDTSV = totalDTSV / impactEventCount
        avgSeverity = totalSeverity / impactEventCount

        if (freeFallEventCount == 0):
            freeFallEventCount = 1
        avgMinTSV = totalMinTSV / freeFallEventCount
        avgSpin = totalSpinDetectedEventCount / freeFallEventCount
    
    if (minTSV == 1000.0):
        minTSV = 0.0
    outputDataList.append([ maxTSV,maxDTSV,avgTSV,avgDTSV,avgSeverity,minTSV,avgMinTSV,avgSpin,1 ])
    #print("ModelInput:", [ maxTSV,maxDTSV,avgTSV,avgDTSV,avgSeverity,minTSV,avgMinTSV,avgSpin])
    return [ maxTSV, maxDTSV, avgTSV, avgDTSV, avgSeverity, minTSV, avgMinTSV, avgSpin]


### Export new DF as an CSV file

In [34]:
def makeOutputDataFrame(folder_name):
    df = pd.DataFrame(outputDataList)
    df.columns = ['maxTSV','maxDTSV','avgTSV','avgDTSV', 'avgSeverity', 'minTSV','avgMinTSV','avgSpin','result']
    dataframe_csv_file_name  = folder_name + '_DataFrame.csv'
    df.to_csv(dataframe_csv_file_name, index=False)

### List of CSV file path

In [35]:
import os

def getAccelerometerDataCsvFilesNamePath(folder_name):
    path_array = []
    dataFolderPath = folder_name + "/"

    dirPath = os.path.join(os.getcwd(), dataFolderPath)
  
    for f in os.listdir(dirPath):
	    if os.path.isfile(os.path.join(dirPath, f)):
              path_array.append("".join([folder_name, "/", f]))

    return path_array

### Calculate TSV DTSV & call detectEvents()

In [36]:
#######################
#        MAIN         #
#######################
def preProcessData(fullFilename):
  # 1. Initializations
  # fullFilename = 'FSXXXX_2017_11_01_18_32_29  - preimpact.csv'  NOT in USE

  # 2. Load raw data
  csv_path = os.path.join(os.getcwd(), fullFilename)
  fileData = pd.read_csv(csv_path)
  fileData["TS"] = fileData["TS"] - fileData["TS"][0]
  fileData["TS"] = fileData["TS"] / 1000000
  numberOfSamples = fileData["TS"].count()
  timeDiffInMs = round(np.median(np.diff(fileData["TS"])))
  print("Timediff median:", timeDiffInMs)

  print('Number of samples:', numberOfSamples)

  # 3. Initialize algorithm parameters
  # 3.1. TSV
  tsv = list()
  dtsv = list()
  for i in range(numberOfSamples):
      tsv.append(np.sqrt(np.square(fileData["X"][i]) + np.square(fileData["Y"][i]) + np.square(fileData["Z"][i])))

  # 3.2. DTSV
  dtsv.append(0)
  for i in range(1, numberOfSamples):
      dtsv.append(tsv[i] - tsv[i - 1])

  # 4. Process data
  print('Interpolation off')
  numberOfInterpolatedSamples = numberOfSamples
  tsInterpolated = copy.deepcopy(fileData["TS"][:])
  xInterpolated = copy.deepcopy(fileData["X"][:])
  yInterpolated = copy.deepcopy(fileData["Y"][:])
  zInterpolated = copy.deepcopy(fileData["Z"][:])
  tsvInterpolated = copy.deepcopy(tsv[:])
  dtsvInterpolated = copy.deepcopy(dtsv[:])

  print('Resampling off')
  numberOfResampledSamples = numberOfInterpolatedSamples
  tsResampled = tsInterpolated[:]
  xResampled = xInterpolated[:]
  yResampled = yInterpolated[:]
  zResampled = zInterpolated[:]
  tsvResampled = tsvInterpolated[:]
  dtsvResampled = dtsvInterpolated[:] 

  # 5. Detect events
  events, noises = detectEvents(tsResampled, tsvResampled, dtsvResampled)
  print("Detected events:", events)
  prepareTFLiteModelInput(events)

### START of execution

In [37]:
def main(folder):
  file_name_list = getAccelerometerDataCsvFilesNamePath(folder)
  for file_name in file_name_list:
    print(file_name)
    preProcessData(file_name)

main('FS')
makeOutputDataFrame('FS')

FS/FSXXXX_2017_11_01_18_41_46  - preimpact.csv
Timediff median: 10
Number of samples: 101
Interpolation off
Resampling off
Detect Event Count: 0
Noise Count: 0
Detected events: []
FS/FSXXXX_2017_11_01_18_34_16  - preimpact.csv
Timediff median: 10
Number of samples: 101
Interpolation off
Resampling off
Area under curve: 0.0
Detect Event Count: 2
Noise Count: 0
Detected events: [[1, 15, 16, 26.249476386769825, 1.8538810045393177, 1], [1, 52, 85, 67.94475051524732, 34.347557779514005, 4]]
6
[1, 15, 16, 26.249476386769825, 1.8538810045393177, 1]
6
[1, 52, 85, 67.94475051524732, 34.347557779514005, 4]
FS/FSXXXX_2017_11_01_18_42_45  - preimpact.csv
Timediff median: 10
Number of samples: 101
Interpolation off
Resampling off
Area under curve: 0.0
Area under curve: 0.0
Detect Event Count: 3
Noise Count: 0
Detected events: [[1, 11, 20, 34.63662275600508, 4.226727855029019, 1], [1, 42, 46, 26.739049349377403, 3.372783334628018, 1], [1, 51, 84, 67.94475051524732, 24.35138593749089, 4]]
6
[1, 11, 2